In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector

In [11]:
def obtain_returns_from_universe():
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
        user="admin",
        password="ubctgquant",
        database="ubctg"
    )

    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Define the start and end dates
    start_date = '2018-01-01'
    end_date = '2021-01-31'
    
    # SQL query to retrieve data from the "Volatility" table between two dates
    sql_query = f"SELECT * FROM Returns WHERE Date BETWEEN '{start_date}' AND '{end_date}'"
    
    # Execute the SQL query
    cursor.execute(sql_query)
    
    # Fetch all rows from the result set
    returns_data = cursor.fetchall()
    
    cursor.close()

    # turn it into a proper Pandas dataframe
    returns_df = pd.DataFrame(returns_data, columns=[i[0] for i in cursor.description])
    # Convert 'Date' column to datetime
    returns_df['Date'] = pd.to_datetime(returns_df['Date'])
    
    # Set the 'Date' column as the index
    returns_df.set_index('Date', inplace=True)

    return returns_df

In [12]:
returns_data = obtain_returns_from_universe()

In [28]:
t = "2018-02-02"

In [29]:
returns_data.loc[t]

,Permno,Ticker,Returns
Date,,,
2018-02-02,10026,JJSF,-0.018346
2018-02-02,10028,DGSE,-0.020619
2018-02-02,10032,PLXS,-0.011076
2018-02-02,10044,RMCF,0.004075
2018-02-02,10051,None,NaN
...,...,...,...
2018-02-02,93427,FN,-0.018838
2018-02-02,93428,None,0.001821
2018-02-02,93429,CBOE,-0.013987
